<a href="https://colab.research.google.com/github/anwojno/CNN/blob/master/new_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#upgrade keras-preprocessing library to the latest verion (multilabel support for flow_from_dataframe)
!pip install --upgrade git+https://github.com/keras-team/keras-preprocessing.git
  
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import ast

from keras.models import Sequential  # initializing nn as a sequence of layers
from keras.layers import Convolution2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense  # fully connected layer
from keras.layers import Dropout
from keras.utils import np_utils # for hard encoding multiclass labels
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array  # for preprocessing training and test images   
from skimage.io import imread  # for loading image for prediction
import numpy as np  # for preprocessing image used for prediction

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.optimizers import Adam

In [0]:
# import csv file (remember to remove last comma from the first row of the csv file)
from google.colab import files

uploaded = files.upload()

Saving Data_Entry_2017.csv to Data_Entry_2017.csv


In [0]:
df = pd.read_csv('/content/data.csv', sep=',', dtype=str)


In [0]:

def str_to_list (classes):
  global labels
  classes = ast.literal_eval(classes)
  labels.update(classes)
  return classes

In [0]:
#contains all the labels names
labels = set()
df['class'] = df['class'].apply(str_to_list)

In [0]:
#counting how many images we downloaded
image_count = 0
image_count = !ls -1 /tmp/images | wc -l
image_count = int(image_count[0])
print(image_count)
print(labels)

15698
{'Infiltration', 'Pneumothorax', 'Nodule', 'Effusion', 'Pneumonia', 'Mass', 'Fibrosis', 'Cardiomegaly', 'No Finding', 'Consolidation', 'Hernia', 'Pleural_Thickening', 'Edema', 'Emphysema', 'Atelectasis'}


In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_dataframe(dataframe = df.truncate(after = 3000), directory = "/tmp/images", x_col='image' ,y_col='class', classes = labels, target_size=(224, 224), color_mode='rgb', class_mode='categorical', batch_size=64)
test_set = test_datagen.flow_from_dataframe(dataframe = df.truncate(before= 3000, after=4000), directory = '/tmp/images', x_col='image', y_col='class', classes = labels, target_size=(224, 224), color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True)

#ResNet50 model with pretrained weights on the ImageNet dataset
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

#constructing classifier from ResNet50
#adding 3 layers to fit output format (15 classes)
classifier = Sequential()
classifier.add(base_model)
classifier.add(AveragePooling2D((7, 7), name='avg_pool'))
classifier.add(Flatten())
classifier.add(Dense(15, activation="sigmoid"))

#here you can freeze ResNet weights or make them trainable
for layer in base_model.layers:
    layer.trainable = True

optimizer = Adam(lr=1e-3)
classifier.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
classifier.summary()


!mkdir /tmp/cb/training_1
checkpoint_path = "/tmp/cb/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
open("/tmp/cb/training_1/cp.ckpt", 'w+')


# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)



classifier.fit_generator(training_set,
                         steps_per_epoch=(training_set.n//training_set.batch_size), #  number of samples of your dataset divided by the batch size
                         epochs=2, ###30
                         validation_data=test_set,
                         validation_steps=(test_set.n//test_set.batch_size),  # number of samples of your validation dataset divided by the batch size
                         callbacks = [cp_callback])  # pass callback to trainin

Found 3001 images belonging to 15 classes.
Found 1001 images belonging to 15 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 15)                30735     
Total params: 23,618,447
Trainable params: 23,565,327
Non-trainable params: 53,120
_________________________________________________________________
mkdir: cannot create directory ‘/tmp/cb/training_1’: File exists
Epoch 1/2


In [0]:
img = load_img('/tmp/images/00001582_006.png', target_size=(224, 224))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 224, 224)
x = x.reshape((1,) + x.shape) 

predictions_single = classifier.predict(x)

In [0]:
print(predictions_single)

training_set.class_indices

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.2747527e-25
  0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]


{'Atelectasis': 0,
 'Cardiomegaly': 1,
 'Consolidation': 2,
 'Edema': 3,
 'Effusion': 4,
 'Emphysema': 5,
 'Fibrosis': 6,
 'Hernia': 7,
 'Infiltration': 8,
 'Mass': 9,
 'No Finding': 10,
 'Nodule': 11,
 'Pleural_Thickening': 12,
 'Pneumonia': 13,
 'Pneumothorax': 14}